In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import warnings

In [ ]:
data = pd.read_csv('EvictionMemphis2020.csv')
variables = ['EvicTotal','Pct_in_pov']
data_selected = data[variables]
data_cleaned = data_selected[(data_selected['EvicTotal'] > 0) 
                             & (data_selected['Pct_in_pov'] > 0)]
print('# of observations (original):', len(data))
print('# of observations (cleaned):', len(data_cleaned))

In [ ]:
#stats for non transformed
descriptive_stats = data_cleaned.describe()
print("Descriptive statistics for the selected variables:")
print(descriptive_stats)

In [ ]:
#skewness for non transformed
skewness_values = data_cleaned.skew()
print("Skewness of the selected variables:")
print(skewness_values)

In [ ]:
def corrfunc(x, y, **kwargs):
    stat, pval = pearsonr(x, y)
    ax = plt.gca() 
    ax.annotate('r = {}'.format(np.round(stat, 2)),  xy = (0.6, 0.1),
                xycoords = ax.transAxes, fontsize = 12, color = 'red') 
g = sns.PairGrid(data_cleaned, diag_sharey = False)
g.map_upper(sns.regplot, scatter_kws = {'alpha': 0.3}) 
g.map_upper(corrfunc)                                  
g.map_diag(sns.histplot, kde = False)                  
g.map_lower(sns.kdeplot, cmap = 'Spectral_r')                       
plt.suptitle('Scatterplot for the variables', 
             y = 1.1)
plt.show()

---

In [ ]:
##stats for log transformed
data_log = np.log(data_cleaned)
data_log = data_log.rename(columns = {'EvicTotal': 'Log_EvicTotal',
                                      'Pct_in_pov': 'Log_Pct_in_pov' })
descriptive_stats = data_log.describe()
print("Descriptive statistics for log variables:")
print(descriptive_stats)

In [ ]:
#skewness for log transformed
skewness_values = data_log.skew()
print("Skewness of the log variables:")
print(skewness_values)

In [ ]:
def corrfunc(x, y, **kwargs):
    stat, pval = pearsonr(x, y)
    ax = plt.gca() 
    ax.annotate('r = {}'.format(np.round(stat, 2)),  xy = (0.6, 0.1),
                xycoords = ax.transAxes, fontsize = 12, color = 'red') 
g = sns.PairGrid(data_log, diag_sharey = False)
g.map_upper(sns.regplot, scatter_kws = {'alpha': 0.3}) 
g.map_upper(corrfunc)                                  
g.map_diag(sns.histplot, kde = False)                  
g.map_lower(sns.kdeplot, cmap = 'Spectral_r')                       
plt.suptitle('Scatterplot for the log variables', 
             y = 1.1)
plt.show()

***

In [ ]:
from IPython.display import HTML

HTML('<span style="color:red;">From here on out I use Log-transformed evictions but non-transformed poverty since it became more skewed when transformed</span>')



In [ ]:
import statsmodels.api as regression

In [ ]:
X = data_cleaned['Pct_in_pov']  
y_log = data_log['Log_EvicTotal']
X = regression.add_constant(X)
model = regression.OLS(y_log, X).fit()
print(model.summary())

In [ ]:
data_combined = pd.merge(data_cleaned[['Pct_in_pov']], data_log[['Log_EvicTotal']], left_index=True, right_index=True)

In [ ]:
#final scatterplot using Log-transformed evictions and non-transformed poverty
def corrfunc(x, y, **kwargs):
    stat, pval = pearsonr(x, y)
    ax = plt.gca() 
    ax.annotate('r = {}'.format(np.round(stat, 2)),  xy = (0.6, 0.1),
                xycoords = ax.transAxes, fontsize = 12, color = 'red') 
g = sns.PairGrid(data_combined, diag_sharey = False)
g.map_upper(sns.regplot, scatter_kws = {'alpha': 0.3}) 
g.map_upper(corrfunc)                                  
g.map_diag(sns.histplot, kde = False)                  
g.map_lower(sns.kdeplot, cmap = 'Spectral_r')                       
plt.suptitle('Scatterplot', 
             y = 1.1)
plt.show()

#final regression using Log-transformed evictions and non-transformed poverty
plt.figure(figsize = (8, 6))
sns.regplot(x = 'Pct_in_pov', y = 'Log_EvicTotal', data = data_combined,
            scatter_kws = {'alpha': 0.5}, line_kws = {'color': 'blue'})
plt.title('Linear Regression: Log_EvicTotal vs Pct_in_pov')
plt.show()

#final regression-standardized-thingy using Log-transformed evictions and non-transformed poverty
y_pred = model.predict(X)
y = data_combined['Log_EvicTotal']
residuals = y - y_pred
standardized_residuals = residuals / np.std(residuals)
plt.figure(figsize = (8, 6))
plt.scatter(y_pred, standardized_residuals, alpha = 0.5)
plt.axhline(y = 0, color='r', linestyle = '--', linewidth = 2)
plt.axhline(y = 2, color='g', linestyle = '--', linewidth = 1) 
plt.axhline(y = -2, color='g', linestyle = '--', linewidth = 1)
plt.title('Standardized vs. Fitted values')
plt.xlabel('Fitted Values (Predicted EvicTotal)')
plt.ylabel('Standardized residuals')
plt.show()